In [154]:
import numpy as np
import pandas as pd
#import scipy
from datetime import date, datetime

In [155]:

users = pd.read_csv(r"D:\dataset\users_cleaned.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")
ratings = pd.read_csv(r"D:\dataset\animelists_cleaned.csv", nrows = 2000000)
af = pd.DataFrame(anime)
rf = pd.DataFrame(ratings)


Generating additional user fetures for data embedding 

In [175]:
#GENERATE USER VECTOR

af = af.loc[af["type"].isin(["TV","Movie"]) & (af["rating"]!="Rx - Hentai")] #only include TV shows and Movies and remove nsfw shows
uf = users.loc[:, ["user_id", "user_completed", "user_days_spent_watching", "gender", "birth_date", "stats_episodes"]]
uf["birth_date"] = pd.to_datetime(uf["birth_date"])

#Only include users who have completed at least 5 anime
uf = uf[uf["user_completed"] >= 5]

#CREATE GENERATION FEATURE
#Convert birth years to generations
def get_birth_year(date):
    birth_year = date.strftime("%Y")
    return int(birth_year)

uf["birth_date"] = uf["birth_date"].apply(get_birth_year)

uf = uf.rename(columns={"birth_date":"generation"})

#Assign generations based on birth year
generations = [
    (uf["generation"] >= 2013) & (uf["generation"] <= 2025),
    (uf["generation"] >= 1995) & (uf["generation"] <= 2012),
    (uf["generation"] >= 1980) & (uf["generation"] <= 1994),
    (uf["generation"] >= 1965) & (uf["generation"] <= 1979)
]

gen_values = [
    "Gen Alpha",
    "Gen Z",
    "Millenials",
    "Gen X"
]
uf["generation"] = np.select(generations, gen_values, default=uf["generation"])

#CREATE EXPERIENCE FEATURE
#Cast days_watched to integers so that users are not missed in conditions below
uf["user_days_spent_watching"] = uf["user_days_spent_watching"].astype("int")
#Assign experience levels based on days watched
user_exp = [
    (uf["user_days_spent_watching"] >= 0) & (uf["user_days_spent_watching"] <= 5),
    (uf["user_days_spent_watching"] >= 6) & (uf["user_days_spent_watching"] <= 50),
    (uf["user_days_spent_watching"] >= 51) & (uf["user_days_spent_watching"] <= 99),
    (uf["user_days_spent_watching"] >= 100)
]

exp_values = [
    "Newbie",
    "Regular",
    "Active",
    "Veteran"
]

uf["user_days_spent_watching"] = np.select(user_exp, exp_values, default=uf["user_days_spent_watching"])
uf = uf.rename(columns={"user_days_spent_watching":"experience"})
uf


#CREATE FAVOURITE GENRE FEATURE

#Only include users who have rated more than 10 anime
num_user_ratings = rf.groupby("username").size() 
limit = num_user_ratings[num_user_ratings > 10].index
usernames = rf["username"].isin(limit)
rf = rf.loc[usernames]

#Create a merged dataframe of ratings and anime dataframes
merge_rf = pd.merge(rf, af, on="anime_id")
merge_rf = merge_rf.loc[:, ["username", "anime_id","title","genre", "my_score","my_watched_episodes", "episodes"]]
#Sort by highest rated anime by each user
merge_rf.set_index("username")
merge_rf = merge_rf.groupby("username").apply(lambda i:i.sort_values(by="my_score", ascending=False))
merge_rf.reset_index(drop=True, inplace=True)
#Only include top 10 highest rated anime per user
merge_rf = merge_rf.groupby("username").head(10)

#Example user
merge_rf.loc[merge_rf["username"]=="karthiga"]
#merge_rf
#get top 3 genres that appear the most

#fav_list = [] list of most popular genres for each user
#put this list in merge with user df

#user_id    |   fav_genre
#account for number of episodes watched
#get most popular genres 
#create the feature
#limitation is that existing users' favourite genres are generated by implication compared to 
# new user who is asked directly


#new features
#Favourite genre(s)
#Perferred anime length

#4. see if any shows appear in multiple people's top 10 lists
#Train set and test set
#Then use lightfm in a new version

#take propensity to rewatch: rewatched/watched (rw)
#take propensity to drop: dropped/watched (dr)
#cosine similarity of all users and movies takes a lot of resources

,user_id,user_completed,experience,gender,generation,stats_episodes
0,2255153,49,Active,Female,Millenials,3391
1,37326,195,Active,Male,Millenials,4903
2,228342,414,Veteran,Male,Millenials,9701
3,327311,5,Regular,Female,Gen Z,697
4,5015094,114,Regular,Male,Gen Z,1847
...,...,...,...,...,...,...
108706,797785,213,Active,Male,Gen Z,3939
108707,1396285,116,Regular,Female,Gen Z,2537
108708,2478991,103,Regular,Female,Gen Z,2869
108709,3975907,239,Active,Male,Gen Z,4469


In [ ]:

#Building a dictionary of all genres - 43
def split(genres):
    return str(genres).split(",")

split_genres = af["genre"].apply(split)
#Updating the set to get the individual genre names in the set instead of the original list of genres
genre_set = set()
for i in split_genres: 
    genre_set.update(i)
result = list(genre_set)

#Removing duplicates with blank spaces
final_result = []
for i in result:
    if i!="nan":
        new = i.replace(" ", "")
        final_result.append(new)

#Removing duplicates now that spaces have been removed
genre_list = list(set(final_result))
print(genre_list)

#COSINE similarity of anime

#check ratings table and find users most loved and hated genres

#anime vector
#filter anime that only have a certain number of ratings
#feature: new-gen/classic/medium
#consider the length of the anime. Ranges: short, medium, long
#consider production studio
#label encoding for normal fields
#one-hot encoding (binary) for genres

#Go through anime_cleaned file
#Merge this file with the users file
#Find all the users that rated each anime
#Find what those users rated those anime and which groups those users belong to
# So you know users from group X liked anime Y "this" much
#Genre: Make each genre its own binary column

#consider number of ratings anime has
#Exclude "hentai genre"
#remember to split the data
#all "currently running" anime show up as having 0 episodes
#filter type to TV only
#give user ability to choose long or short animes


#af = pd.DataFrame(anime)
#af = af.loc[pd.isna(af["title_english"]) == False,:]
#af



['School', 'Parody', 'Police', 'Vampire', 'Seinen', 'ShoujoAi', 'MartialArts', 'Dementia', 'Shounen', 'Harem', 'Mecha', 'Historical', 'ShounenAi', 'Kids', 'Ecchi', 'Josei', 'Space', 'Sports', 'Action', 'Comedy', 'Demons', 'Romance', 'Samurai', 'Fantasy', 'Supernatural', 'Game', 'Horror', 'Mystery', 'Sci-Fi', 'Military', 'Shoujo', 'SuperPower', 'Cars', 'Magic', 'Psychological', 'Music', 'SliceofLife', 'Thriller', 'Drama', 'Adventure']
